# Mot back-end demonstration

In this notebook we demonstrate the most basic of all cold atoms systems a MOT and most helpful for tests of basic functionalities of new devices. It is a cloud of atoms that is cooled down to a few micro Kelvin. The atoms are trapped in a magneto-optical trap (MOT) and can be used for further experiments. The MOT is the starting point for many experiments in cold atoms physics. In this notebook we will demonstrate how to use the MOT backend of the qlued platform. It allows us to demonstrate the basic functionality of the platform and the two gates that are always required `load` and `measure`. 

The communication with the backend happens through the four url

1. '.../api/v2/alqor_mot_simulator/get_config/' 
2. '.../api/v2/alqor_mot_simulator/post_job/'
3. '.../api/v2/alqor_mot_simulator/get_job_status/'
4. '.../api/v2/alqor_mot_simulator/get_job_result/'

An interactive documentation can be also found directly online under the [docs](https://qlued.alqor.io/api/v2/docs).

You can use this tutorial in two ways. For testing of local deployments of `qlued` or the [hosted version](https://qlued.alqor.io) provided by Alqor.

## Optional: Preparation for local deployment

Some first things that you have to do:

- Apply any migrations to your local server `python manage.py migrate`
- Start the server in a tab as `python manage.py runserver`
- You created an account and saved this in the local `credentials.py` file

## Get the configuration

In a first step, we will try to see what are the available backends and what are the capabilities. This can be done through the json API endpoint `get_config` for the `alqor_mot_simulator` backend.

In [1]:
import requests
import json
from pprint import pprint

In [2]:
from credentials_v2 import username, token

In [3]:
host_url = "https://qlued.alqor.io/"
device_name = "alqor_mot_simulator"

#host_url = "http://localhost:8000/"
#device_name = "localtest_mot_simulator"

url_prefix = host_url + f"api/v2/{device_name}/"

In [4]:
url = url_prefix + "get_config"
pprint(url)
r = requests.get(url, params={"username": username, "token": token})
r_dict = json.loads(r.text)
pprint(r_dict)

'https://qlued.alqor.io/api/v2/alqor_mot_simulator/get_config'
{'backend_name': 'alqor_mot_simulator',
 'backend_version': '0.1',
 'basis_gates': [],
 'cold_atom_type': 'spin',
 'conditional': False,
 'coupling_map': 'linear',
 'description': 'Setup of an atomic mot.',
 'display_name': 'mot',
 'dynamic_reprate_enabled': False,
 'gates': [],
 'local': False,
 'max_experiments': 1000,
 'max_shots': 1000000,
 'memory': True,
 'n_qubits': 1,
 'num_species': 1,
 'open_pulse': False,
 'simulator': True,
 'supported_instructions': ['barrier', 'measure', 'load'],
 'url': 'https://qlued.alqor.io/api/v2/alqor_mot_simulator/',
 'wire_order': 'interleaved'}


The above json gives us all the necessary information. The system has one wire, in other words one MOT. For this wire we have implemented the following instructions:
 
 - `load` which adds a Fermion to the wire.
 - `measure` which reads out the occupation.

## Load gate

It is now time to try a simple sequence, where we load the MOT.

In [5]:
job_payload = {
    "experiment_0": {
        "instructions": [
            ("load", [0], [20]),
            ("measure", [0], []),
        ],
        "num_wires": 1,
        "shots": 2,
        "wire_order": "interleaved",
    },
}

Now, that we set up the instruction, we can submit it via the `post_job` endpoint.

In [15]:
url = url_prefix + "post_job"

pprint(url)
job_response = requests.post(
    url,
    json={"job": json.dumps(job_payload), "username": username, "token": token},
)
job_id = (job_response.json())["job_id"]
print(job_response.json())

'https://qlued.alqor.io/api/v2/alqor_mot_simulator/post_job'
{'job_id': '0e4200b3ab894dde936f1716', 'status': 'INITIALIZING', 'detail': 'Got your json.', 'error_message': 'None'}


The simulator has now put the job into the queue and we will have to wait until the calculation is finished. To see its test the job status through `get_job_status`.

In [12]:
url = url_prefix + "get_job_status"

status_response = requests.get(
    url, params={"job_id": job_id, "username": username, "token": token}
)

print(status_response.text)

{"job_id": "1730f8085c1140ec96b66d68", "status": "DONE", "detail": "None; Passed json sanity check; Compilation done. Shots sent to solver.", "error_message": "None"}


As we can see the job is finished, so let us see the results the `get_job_result` endpoint.

In [13]:
url = url_prefix + "get_job_result"

result_response = requests.get(
    url, params={"job_id": job_id, "username": username, "token": token}
)

pprint(result_response.json())

{'backend_name': 'alqor_mot_simulator',
 'backend_version': '0.1',
 'display_name': 'mot',
 'header': {},
 'job_id': '1730f8085c1140ec96b66d68',
 'qobj_id': None,
 'results': [{'data': {'memory': ['20', '20']},
              'header': {'extra metadata': 'text', 'name': 'experiment_0'},
              'shots': 2,
              'success': True}],
 'status': 'finished',
 'success': True}


As you can see the MOT was twice loaded with twenty atoms.

## Conclusion

We have seen how to use the MOT backend of the qlued platform. It allows us to demonstrate the basic functionality of the platform and the two gates that are always required `load` and `measure`.